In [15]:
pip install lxml

   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB 245.8 kB/s eta 0:00:16
    --------------------------------------- 0.1/3.9 MB 393.8 kB/s eta 0:00:10
   - -------------------------------------- 0.1/3.9 MB 599.1 kB/s eta 0:00:07
   - -------------------------------------- 0.2/3.9 MB 748.1 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/3.9 MB 846.9 kB/s eta 0:00:05
   --- ------------------------------------ 0.3/3.9 MB 999.9 kB/s eta 0:00:04
   --- ------------------------------------ 0.4/3.9 MB 1.1 MB/s eta 0:00:04
   ---- ----------------------------------- 0.4/3.9 MB 1.1 MB/s eta 0:00:04
   ---- ----------------------------------- 0.5/3.9 MB 1.2 MB/s eta 0:00:03
   ----- -------------------

In [465]:
import pandas as pd
import geopandas as gpd
import json
import re
import os

In [20]:
filepath = 'C:/Smart/Powershell/queuebackup/2024-02-21-Backup.xml'
#with open(filepath,"r") as f:
df = pd.read_xml(filepath)
#
#    xml_string=file.read()
#    python_dict=xmltodict.parse(xml_string)
    #file.close()

In [21]:
df.head()

nr                                  uuid     type  \
0   0  7cecaada-230d-4078-a5ce-f9faeaf5835d  JSON_CT   
1   0  2b9c011a-c6b2-4c66-9144-dc04d9a1793e  JSON_CT   
2   0  0fbc9c42-4148-4b23-b83d-143745c2eb90  JSON_CT   
3   0  38b49047-95af-4118-9da5-afd675a3cceb  JSON_CT   
4   0  85886d38-5662-4d7b-ac43-bec0bc02f10d  JSON_CT   

                                         name  \
0  Manual Upload 2024_02_21_19_38_37_716.json   
1  Manual Upload 2024_02_21_19_38_25_558.json   
2  Manual Upload 2024_02_21_19_32_25_523.json   
3  Manual Upload 2024_02_21_18_18_08_857.json   
4  Manual Upload 2024_02_21_18_18_07_948.json   

                              caName    status     lastModifiedDate  \
0  Lion Monitoring Namibia [LIONSNA]  COMPLETE  21.02.2024 18:39:56   
1  Lion Monitoring Namibia [LIONSNA]  COMPLETE  21.02.2024 18:39:54   
2  Lion Monitoring Namibia [LIONSNA]  COMPLETE  21.02.2024 18:35:28   
3  Lion Monitoring Namibia [LIONSNA]  COMPLETE  21.02.2024 18:41:01   
4  Lion Monitoring Namibia [LIONSNA]  COMPLETE  21.02.2024 18:41:00   

          uploadedDate     uploadedBy                      conservationArea  \
0  21.02.2024 18:38:38  oliver.adolph  a2516167-3da8-440e-b56b-6f68c2f11d53   
1  21.02.2024 18:38:26  oliver.adolph  a2516167-3da8-440e-b56b-6f68c2f11d53   
2  21.02.2024 18:32:26  oliver.adolph  a2516167-3da8-440e-b56b-6f68c2f11d53   
3  21.02.2024 17:18:09  oliver.adolph  a2516167-3da8-440e-b56b-6f68c2f11d53   
4  21.02.2024 17:18:08  oliver.adolph  a2516167-3da8-440e-b56b-6f68c2f11d53   

  LoadStatus                                               file  \
0         Ok  {"features": [{\n    "geometry": {\n        "c...   
1         Ok  {"features": [{\n    "geometry": {\n        "c...   
2         Ok  {\n  "features": [\n    {\n      "geometry": {...   
3         Ok  {"features": [{\n    "geometry": {\n        "c...   
4         Ok  {"features": [{\n    "geometry": {\n        "c...   

                               deviceId  
0  66f08aa6-684a-b7fc-0000-000000000000  
1  66f08aa6-684a-b7fc-0000-000000000000  
2  66f08aa6-684a-b7fc-0000-000000000000  
3  66f08aa6-684a-b7fc-0000-000000000000  
4  66f08aa6-684a-b7fc-0000-000000000000

In [219]:
#df2 = df.where(df['uploadedDate'] > '2024-01-01').where(df['file'] != None)
df2 = df.where(df['file'] != None)


In [220]:
df2 = df2.dropna(how='all')

In [221]:
#df2['file']

0       {"features": [{\n    "geometry": {\n        "c...
1       {"features": [{\n    "geometry": {\n        "c...
2       {\n  "features": [\n    {\n      "geometry": {...
3       {"features": [{\n    "geometry": {\n        "c...
4       {"features": [{\n    "geometry": {\n        "c...
                              ...                        
5299    {"type": "FeatureCollection", "features": [{\n...
5300    {"type": "FeatureCollection", "features": [{\n...
5301    {"type": "FeatureCollection", "features": [{\n...
5302    {"type": "FeatureCollection", "features": [{\n...
5303    {"type": "FeatureCollection", "features": [{\n...
Name: file, Length: 5304, dtype: object

In [359]:
dataqueue = []
for j in df2['file']:
    if j:
        data = json.loads(j)
        dataqueue.append(gpd.GeoDataFrame.from_features(data))
    #print(gdf.head())

In [362]:
patrols = []
for dq in dataqueue: #[0:10000]:
    if 'sighting' in dq.columns:
        #print(dq.columns)
        # Drop all rows where there is no sighting 
        dq2 = dq.dropna(subset = ['sighting'])
        sighting = dq2.sighting
        deviceId = dq2.deviceId.min()
        #print(deviceId)
        for s in sighting: 
            try:
                if s["SMART_ObservationType"] == "NewPatrol":
                    s['deviceId'] = deviceId
                    #print(s['deviceId'])
                    patrols.append(s)
                    
            except Exception as e:
                print("Error:")
                print(e)
                            
#patrols

In [363]:
sonnet_uuid = 'e:cd6e38f8a59b47b0bb2c5d15d10f50c3'
esau_uuid = 'e:44f58aa330124c2baa4e8903166a5a12'
e_uuid = sonnet_uuid

In [404]:
dfPatrols = pd.DataFrame(patrols)
dfPatrols['patrol_start'] = pd.to_datetime(dfPatrols['SMART_PatrolStartDate'] + ' ' + dfPatrols['SMART_PatrolStartTime'])
#dfPatrols.rename(columns={c: c.replace(':', '_') for c in dfPatrols.columns}, inplace=True)
#print(dfPatrols.columns)
#patrols[0]['e:03cd921d6a944e328ca28cf0ef7c0cee']
#dfPatrols.dropna() #subset = ['e:44f58aa330124c2baa4e8903166a5a12'])
#if 'e:44f58aa330124c2baa4e8903166a5a12' in dfPatrols.columns:
    #print('Esau')
esau = dfPatrols.where(dfPatrols[e_uuid]).dropna(how='all') #.where(df['SMART_PatrolStartDate'].dt >= '2024-01-01') #.dropna()
esau.groupby(['deviceId','patrol_start',])['SMART_PatrolID'].count()
#dfPatrols.columns
#esau.groupby(['deviceId'])['SMART_PatrolID'].count()
#['e:44f58aa330124c2baa4e8903166a5a12'].count() #.dropna(how='all')
#dfPatrols.where(dfPatrols['e:4e69110510a84ce7ab367a91d574f0b5']).dropna(how='all')
#dfPatrols.where(dfPatrols['SMART_PatrolLegCounter'] >= 0).reindex().dropna(how='all')


deviceId                          patrol_start       
ba66c7d156a3f4e00000000000000000  2023-12-08 17:01:28    1
                                  2023-12-10 18:58:46    1
                                  2023-12-12 09:11:07    1
                                  2023-12-13 10:01:28    1
                                  2023-12-14 06:22:25    1
                                  2023-12-14 16:10:41    1
                                  2023-12-15 08:17:51    1
                                  2023-12-15 11:34:53    1
                                  2023-12-16 06:24:59    1
                                  2023-12-16 06:26:41    1
                                  2023-12-19 15:03:51    1
                                  2023-12-19 20:37:24    1
                                  2023-12-20 07:05:05    1
                                  2023-12-21 07:49:51    1
                                  2023-12-22 08:05:40    1
                                  2023-12-25 08:07:39    1
  

In [405]:
c = dfPatrols.where(dfPatrols['SMART_PatrolStartDate'] >= '2024-01-20').count() # ['e:44f58aa330124c2baa4e8903166a5a12']
#dfPatrols.count()


In [406]:
c

SMART_Comments                        274
SMART_DataType                        780
SMART_DefaultAttributeValues          780
SMART_DefaultPatrolValues             780
SMART_Leader                          780
                                     ... 
e:ed07ede1cedb40eeba590e9d5cd9c646     31
e:d7538fb3db9f4dbba5c9f53578d05eea      0
e:8ea9ca637c2545d58631f0716bc679a7      0
e:6d4bb8427f1949a9835526e631f05756      0
patrol_start                          780
Length: 69, dtype: int64

In [407]:
#c.filter(regex='^e:44f', axis=0)
c.filter(regex=e_uuid, axis=0)

e:cd6e38f8a59b47b0bb2c5d15d10f50c3    11
dtype: int64

In [411]:
dfPatrols['SMART_PatrolStartDate'] = pd.to_datetime(dfPatrols['SMART_PatrolStartDate'])
esau = dfPatrols.groupby(pd.Grouper(key='SMART_PatrolStartDate', freq='M')).count().filter(regex='^e:', axis=1)[e_uuid]
esau

SMART_PatrolStartDate
2020-07-31     0
2020-08-31     0
2020-09-30     0
2020-10-31     0
2020-11-30     0
2020-12-31     0
2021-01-31     0
2021-02-28     0
2021-03-31     0
2021-04-30     0
2021-05-31     0
2021-06-30     0
2021-07-31     0
2021-08-31     0
2021-09-30     0
2021-10-31     0
2021-11-30     0
2021-12-31     0
2022-01-31     0
2022-02-28     0
2022-03-31     0
2022-04-30     0
2022-05-31     0
2022-06-30     0
2022-07-31     0
2022-08-31     0
2022-09-30     0
2022-10-31     0
2022-11-30     0
2022-12-31     0
2023-01-31     0
2023-02-28     0
2023-03-31     0
2023-04-30     0
2023-05-31     0
2023-06-30     0
2023-07-31     0
2023-08-31     0
2023-09-30     0
2023-10-31     0
2023-11-30     0
2023-12-31    22
2024-01-31     6
2024-02-29     5
Freq: M, Name: e:cd6e38f8a59b47b0bb2c5d15d10f50c3, dtype: int64

In [410]:
x= esau.resample(rule = 'M').sum()

In [331]:
x

SMART_PatrolStartDate
2020-07-31    0
2020-08-01    0
2020-08-02    0
2020-08-03    0
2020-08-04    0
             ..
2024-02-16    0
2024-02-17    1
2024-02-18    1
2024-02-19    2
2024-02-20    1
Freq: D, Name: e:cd6e38f8a59b47b0bb2c5d15d10f50c3, Length: 1300, dtype: int64

In [332]:
#x.index

DatetimeIndex(['2020-07-31', '2020-08-01', '2020-08-02', '2020-08-03',
               '2020-08-04', '2020-08-05', '2020-08-06', '2020-08-07',
               '2020-08-08', '2020-08-09',
               ...
               '2024-02-11', '2024-02-12', '2024-02-13', '2024-02-14',
               '2024-02-15', '2024-02-16', '2024-02-17', '2024-02-18',
               '2024-02-19', '2024-02-20'],
              dtype='datetime64[ns]', name='SMART_PatrolStartDate', length=1300, freq='D')

In [480]:
logfilepath = 'C:/Smart/Support/Lionrangernaconnect/2024-02-25-Sonnet-BugReport/Log_1.0.478.txt'
log_patrols = []
keep_phrases = ["Patrol start success"] #, "Patrol stop success"]
with open(logfilepath) as f:
    f = f.readlines()
for line in f:
    for phrase in keep_phrases:
        if phrase in line:
            log_patrol = dict()
            match = re.findall("^\[(.*)\].*QDateTime\((.*) CAT.*", line)
            
            if match:
                log_patrol['datetime'] = match[0][0]
                log_patrol['log'] = line
                #print(match)
                log_patrols.append(log_patrol)
            break

dfLogPatrols = pd.DataFrame(log_patrols)
dfLogPatrols['datetime'] = pd.to_datetime(dfLogPatrols['datetime'])
#dfLogPatrols

In [481]:

result =dfLogPatrols.where(dfLogPatrols['datetime'] >= '2024-01-20').groupby(pd.Grouper(key='datetime', freq='M')).count()
result
#grpdfLogPatrols = dfLogPatrols.groupby('datetime')
#grpdfLogPatrols.resample(rule='D')

log
datetime                      
2024-01-31 00:00:00+02:00    4
2024-02-29 00:00:00+02:00   26

In [420]:
dfPatrols.where(dfPatrols['SMART_PatrolStartDate'] >= '2024-01-20').groupby('deviceId')['SMART_PatrolID'].count()

deviceId
00bf8d51bdbe088c0000000000000000    13
045de147ea1ca19d0000000000000000    10
0f10be73fcbd75e00000000000000000    13
141e341087797eb50000000000000000    16
1d9fbe36b931c18c0000000000000000     5
23c3d9733ac5b9e40000000000000000    14
249eccf8d10bddae0000000000000000    11
26de50b7ff4c5cec0000000000000000    11
38a5ff713844296f0000000000000000    16
38f99af77e564aff0000000000000000    12
3d485d1d0e90d8f10000000000000000    18
407a59fe2de92c1c0000000000000000    11
4100f4c00a3fb9020000000000000000    10
41e803f10085b3580000000000000000    16
4814fc89b4e51e930000000000000000    15
4e80c8d70a9108680000000000000000     4
5336785f4e7c93180000000000000000    14
55805b320a9088950000000000000000    13
5aa7a376845bd83a0000000000000000    11
5c8b8f0b7130ea760000000000000000    11
6029018529d0f4940000000000000000     1
64ebcb00cce107600000000000000000    16
65123b42df7387750000000000000000    16
66f08aa6684ab7fc0000000000000000    15
682db27fb03b5ee40000000000000000     4
6a42853ee0a7d331

In [421]:
# Devices with more patrols in the dataqueue compared to the database
deviceid = ['da9da87cec9d64ee0000000000000000',
            'cadc3737130ba6700000000000000000',
            '64ebcb00cce107600000000000000000',
            '38a5ff713844296f0000000000000000',
            '0f10be73fcbd75e00000000000000000']

In [431]:
dfPatrols.where((dfPatrols['deviceId'] == deviceid[0]) & (dfPatrols['patrol_start'] >= '2024-01-20')).dropna(how='all').sort_values('SMART_PatrolID')

SMART_Comments SMART_DataType SMART_DefaultAttributeValues  \
669            NaN         patrol                           {}   
656            NaN         patrol                           {}   
668            NaN         patrol                           {}   
658                        patrol                           {}   
664            NaN         patrol                           {}   
663            NaN         patrol                           {}   
665            NaN         patrol                           {}   
662            NaN         patrol                           {}   
661            NaN         patrol                           {}   
657            NaN         patrol                           {}   
666            NaN         patrol                           {}   
659            NaN         patrol                           {}   
660            NaN         patrol                           {}   
667            NaN         patrol                           {}   

    SMART_DefaultPatrolValues                        SMART_Leader  \
669                        {}  e:2ae644c460e84908aaecd9aa7666e102   
656                        {}  e:2ae644c460e84908aaecd9aa7666e102   
668                        {}  e:2ae644c460e84908aaecd9aa7666e102   
658                        {}  e:731dd78506254dc19d60144cd6cec554   
664                        {}  e:2ae644c460e84908aaecd9aa7666e102   
663                        {}  e:2ae644c460e84908aaecd9aa7666e102   
665                        {}  e:2ae644c460e84908aaecd9aa7666e102   
662                        {}  e:2ae644c460e84908aaecd9aa7666e102   
661                        {}  e:2ae644c460e84908aaecd9aa7666e102   
657                        {}  e:731dd78506254dc19d60144cd6cec554   
666                        {}  e:2ae644c460e84908aaecd9aa7666e102   
659                        {}  e:2ae644c460e84908aaecd9aa7666e102   
660                        {}  e:2ae644c460e84908aaecd9aa7666e102   
667                        {}  e:2ae644c460e84908aaecd9aa7666e102   

                           SMART_Mandate SMART_NewWaypoint  SMART_ObsCounter  \
669  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
656  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
668  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
658  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
664  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
663  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
665  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
662  pm:252d31de97b44765b95fa2dd6fcc59bc              True               1.0   
661  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
657  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
666  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
659  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
660  pm:f09da0839f6349dc9aec25f9feaaf0db              True               1.0   
667  pm:252d31de97b44765b95fa2dd6fcc59bc              True               1.0   

    SMART_ObservationType                    SMART_PatrolID  ...  \
669             NewPatrol  1a8d6b44c68d487999a5fc69629c3a1c  ...   
656             NewPatrol  28ee904aea384c4497fd4c2ac1d1ae8a  ...   
668             NewPatrol  2d87e14d2c514562a1580ff0935975e3  ...   
658             NewPatrol  33d0b74f56d4468b839da7ff67efdc1a  ...   
664             NewPatrol  3c9848c0b7484464bf0be4cfb870a519  ...   
663             NewPatrol  455b9c648a2f485fad0ee78c87b72126  ...   
665             NewPatrol  64aa6d7f5caf4e679a2c610e0a1050b7  ...   
662             NewPatrol  7490412ac8924fafbd2e5cdb005f76b6  ...   
661             NewPatrol  7a3c93601e514b2faeb531d0df01339b  ...   
657             NewPatrol  9adaea3e08c349f5bea72f5df231288f  ...   
666             NewPatrol  9

In [ ]:
# Missing Patrol-IDs
missing_patrol_ids =['7490412ac8924fafbd2e5cdb005f76b6', '9e3cdbef85fd4e53998f3c2cad048e17', 'f76184727539436980aa2250029ba2e4']

In [471]:
#filepath = 'C:/Users/OliverAdolph/OneDrive - AVG Vermögensverwaltung GmbH/Desktop/CTProjects/ExportData/CTDATA_2024-02-26T15_14_50_Patrol-0001.json'
dfRecovered = gpd.GeoDataFrame()
path = 'C:/Users/OliverAdolph/OneDrive - AVG Vermögensverwaltung GmbH/Desktop/CTProjects/ExportData/'
dir_list = os.listdir(path)
for filepath in dir_list:
    #gpd.GeoDataFrame.concat(dfRecovered, gpd.GeoDataFrame.from_file(filepath))
    #dfRecovered.append(gpd.GeoDataFrame.from_file(filepath))
    dfRecovered = pd.concat([dfRecovered, gpd.GeoDataFrame.from_file(path + filepath)])
    #dfRecovered = gpd.GeoDataFrame.from_file(filepath)

In [479]:
dataqueue2 = []
for r in dfRecovered["sighting"]:
    if r:
        dataqueue2.append(r)
dfr = pd.DataFrame(dataqueue2)
dfr['SMART_PatrolStartDate'] = pd.to_datetime(dfr['SMART_PatrolStartDate'])
dfr.where((dfr['SMART_ObservationType'] == 'NewPatrol') & (dfr['SMART_PatrolStartDate'] >= '2024-01-20')).dropna(how='all').groupby(pd.Grouper(key='SMART_PatrolStartDate', freq='M'))['SMART_PatrolID'].count()

SMART_PatrolStartDate
2024-01-31     4
2024-02-29    24
Freq: M, Name: SMART_PatrolID, dtype: int64